# Exercise 1 - Localize objects

- Develop an application that localizes an object in a scene, even when the object is partially
occluded.
- Test different feature point detectors and different parameters for matching the points, namely,
the value of the Lowe’s ratio test and the influence of using the RANSAC method or not.
- Test with different objects and scenes; test also with non-planar objects.

- Develop an application that localizes an object in a scene, even when the object is partially occluded (fig. 2).
- Test different feature point detectors and different parameters for matching the points, namely, the value of the Lowe’s ratio test and the influence of using the RANSAC method or not.
- Test with different objects and scenes; test also with non-planar objects.


#### Imports and Setup

In [35]:
from __future__ import print_function
import cv2 as cv
import numpy as np


img_object = cv.imread("../../images/box.png", cv.IMREAD_GRAYSCALE)
img_scene = cv.imread("../../images/scene_box.png", cv.IMREAD_GRAYSCALE)
if img_object is None or img_scene is None:
    print('Could not open or find the images!')
    exit(0)



#### Approach 1 - SIFT + FLANN

This approach uses SIFT, for feature detection and description, and FLANN for feature matching.

In [36]:
#Step 1: Detect the keypoints using SIFT Detector, compute the descriptors

sift = cv.SIFT_create()
# find the keypoints and descriptors with SIFT
keypoints_obj, descriptors_obj = sift.detectAndCompute(img_object,None)
keypoints_scene, descriptors_scene = sift.detectAndCompute(img_scene,None)


#Step 2: Matching descriptor vectors with a FLANN based matcher
matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_matches = matcher.knnMatch(descriptors_obj, descriptors_scene, 2)
#-- Filter matches using the Lowe's ratio test
ratio_thresh = 0.75
good_matches = []
for m,n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)




#-- Draw matches
img_matches = np.empty((max(img_object.shape[0], img_scene.shape[0]), img_object.shape[1]+img_scene.shape[1], 3), dtype=np.uint8)
cv.drawMatches(img_object, keypoints_obj, img_scene, keypoints_scene, good_matches, img_matches, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)


#Step 3: Localize the object
obj = np.empty((len(good_matches),2), dtype=np.float32)
scene = np.empty((len(good_matches),2), dtype=np.float32)

for i in range(len(good_matches)):
    #-- Get the keypoints from the good matches
    obj[i,0] = keypoints_obj[good_matches[i].queryIdx].pt[0]
    obj[i,1] = keypoints_obj[good_matches[i].queryIdx].pt[1]
    scene[i,0] = keypoints_scene[good_matches[i].trainIdx].pt[0]
    scene[i,1] = keypoints_scene[good_matches[i].trainIdx].pt[1]

H, _ =  cv.findHomography(obj, scene, cv.RANSAC) # ---- RANSAC



#-- Get the corners from the image_1 ( the object to be "detected" )
obj_corners = np.empty((4,1,2), dtype=np.float32)
obj_corners[0,0,0] = 0
obj_corners[0,0,1] = 0
obj_corners[1,0,0] = img_object.shape[1]
obj_corners[1,0,1] = 0
obj_corners[2,0,0] = img_object.shape[1]
obj_corners[2,0,1] = img_object.shape[0]
obj_corners[3,0,0] = 0
obj_corners[3,0,1] = img_object.shape[0]
scene_corners = cv.perspectiveTransform(obj_corners, H)
#Step 5: Draw lines between the corners (the mapped object in the scene - image_2 )
cv.line(img_matches, (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])),\
    (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])),\
    (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])),\
    (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])),\
    (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])), (0,255,0), 4)

#-- Show detected matches
cv.imshow('SIFT + FLANN', img_matches)
cv.waitKey(0)
cv.destroyAllWindows()

#### Approach 2 - ORB + Brute Force

This approach uses ORB for feature detection and description, and Brute Force, with hamming distance, for feature matching.

In [37]:
#Step 1: Detect the keypoints using ORB Detector, compute the descriptors

orb = cv.ORB_create()

# find the keypoints and descriptors with ORB

keypoints_obj, descriptors_obj = orb.detectAndCompute(img_object,None)
keypoints_scene, descriptors_scene = orb.detectAndCompute(img_scene,None)

matcher = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True) # used for ORB
knn_matches = matcher.match(descriptors_obj,descriptors_scene)

#Optional, select the top 10% closest matches (most effective)
# top_matches = int(0.1 * len(knn_matches)) # select the top 10% matches
# good_matches = sorted(knn_matches, key = lambda x:x.distance)[:top_matches]


#-- Filter matches using the Lowe's ratio test

ratio_thresh = 0.8
good_matches = []
for m in knn_matches:
    if m.distance < ratio_thresh * knn_matches[1].distance:
        good_matches.append(m)



#-- Draw matches
img_matches = np.empty((max(img_object.shape[0], img_scene.shape[0]), img_object.shape[1]+img_scene.shape[1], 3), dtype=np.uint8)
cv.drawMatches(img_object, keypoints_obj, img_scene, keypoints_scene, good_matches, img_matches, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)


#Step 3: Localize the object
obj = np.empty((len(good_matches),2), dtype=np.float32)
scene = np.empty((len(good_matches),2), dtype=np.float32)

for i in range(len(good_matches)):
    #-- Get the keypoints from the good matches
    obj[i,0] = keypoints_obj[good_matches[i].queryIdx].pt[0]
    obj[i,1] = keypoints_obj[good_matches[i].queryIdx].pt[1]
    scene[i,0] = keypoints_scene[good_matches[i].trainIdx].pt[0]
    scene[i,1] = keypoints_scene[good_matches[i].trainIdx].pt[1]

H, _ =  cv.findHomography(obj, scene, cv.RANSAC) # ---- RANSAC



#-- Get the corners from the image_1 ( the object to be "detected" )
obj_corners = np.empty((4,1,2), dtype=np.float32)
obj_corners[0,0,0] = 0
obj_corners[0,0,1] = 0
obj_corners[1,0,0] = img_object.shape[1]
obj_corners[1,0,1] = 0
obj_corners[2,0,0] = img_object.shape[1]
obj_corners[2,0,1] = img_object.shape[0]
obj_corners[3,0,0] = 0
obj_corners[3,0,1] = img_object.shape[0]
scene_corners = cv.perspectiveTransform(obj_corners, H)
#Step 5: Draw lines between the corners (the mapped object in the scene - image_2 )
cv.line(img_matches, (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])),\
    (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])),\
    (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])),\
    (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])),\
    (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])), (0,255,0), 4)
#-- Show detected matches
cv.imshow('ORB + Brute Force', img_matches)
cv.waitKey(0)
cv.destroyAllWindows()

#### Approach 3 - BRISK + Brute Force

This approach uses BRISK for feature detection and description, and Brute Force, with hamming distance, for feature matching.

In [38]:
#Step 1: Detect the keypoints using BRISK Detector, compute the descriptors


brisk = cv.BRISK_create()

# find the keypoints and descriptors with BRISK
keypoints_obj, descriptors_obj = brisk.detectAndCompute(img_object,None)
keypoints_scene, descriptors_scene = brisk.detectAndCompute(img_scene,None)

matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_BRUTEFORCE) # used for BRISK
knn_matches = matcher.match(descriptors_obj,descriptors_scene)

#Optional, select the top 5% closest matches (most effective)
# top_matches = int(0.05 * len(knn_matches)) # select the top 10% matches
# good_matches = sorted(knn_matches, key = lambda x:x.distance)[:top_matches]

#-- Filter matches using the Lowe's ratio test
ratio_thresh = 0.85
good_matches = []
for m in knn_matches:
    if m.distance < ratio_thresh * knn_matches[0].distance:
        good_matches.append(m)



#-- Draw matches
img_matches = np.empty((max(img_object.shape[0], img_scene.shape[0]), img_object.shape[1]+img_scene.shape[1], 3), dtype=np.uint8)
cv.drawMatches(img_object, keypoints_obj, img_scene, keypoints_scene, good_matches, img_matches, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)


#Step 3: Localize the object
obj = np.empty((len(good_matches),2), dtype=np.float32)
scene = np.empty((len(good_matches),2), dtype=np.float32)

for i in range(len(good_matches)):
    #-- Get the keypoints from the good matches
    obj[i,0] = keypoints_obj[good_matches[i].queryIdx].pt[0]
    obj[i,1] = keypoints_obj[good_matches[i].queryIdx].pt[1]
    scene[i,0] = keypoints_scene[good_matches[i].trainIdx].pt[0]
    scene[i,1] = keypoints_scene[good_matches[i].trainIdx].pt[1]

H, _ =  cv.findHomography(obj, scene, cv.RANSAC) # ---- RANSAC



#-- Get the corners from the image_1 ( the object to be "detected" )
obj_corners = np.empty((4,1,2), dtype=np.float32)
obj_corners[0,0,0] = 0
obj_corners[0,0,1] = 0
obj_corners[1,0,0] = img_object.shape[1]
obj_corners[1,0,1] = 0
obj_corners[2,0,0] = img_object.shape[1]
obj_corners[2,0,1] = img_object.shape[0]
obj_corners[3,0,0] = 0
obj_corners[3,0,1] = img_object.shape[0]
scene_corners = cv.perspectiveTransform(obj_corners, H)
#Step 5: Draw lines between the corners (the mapped object in the scene - image_2 )
cv.line(img_matches, (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])),\
    (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])),\
    (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])),\
    (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])),\
    (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])), (0,255,0), 4)
#-- Show detected matches
cv.imshow('Brisk', img_matches)
cv.waitKey(0)
cv.destroyAllWindows()